In [65]:
import os
import time
from sklearn.tree import DecisionTreeClassifier
from numpy import array
import pandas as pd
import numpy as np
from numpy import array
from numpy import zeros
from numpy import shape
os.chdir("D:\\practicespace\\github\\Help_Navigate_Robots\\dataset")

### 将数据进行join操作，然后按照分类进行排序（用于后面统计各个分类数量）

In [66]:
def getSampleData():
    # 读入数据，加载到Pandas里面
    start_time = time.time()
    pd_x = pd.read_csv("x_train.csv")
    end_time = time.time()
    #print("pandas阅读csv文件耗时: ", end_time - start_time)
    pd_y = pd.read_csv("y_train.csv")
    pd_data = pd_x.set_index("series_id").join(pd_y.set_index("series_id"))
    print("pd_data样本: \n", pd_data.head(3))
    # 对数据列进行归一化
    max_min_scalar = lambda x: (x - np.min(x))/(np.max(x) - np.min(x))
    normed_Z = pd_data[["linear_acceleration_Z"]].apply(max_min_scalar) # 注意这里一定是套了两层中括号
    normed_Y = pd_data[["linear_acceleration_Y"]].apply(max_min_scalar)
    pd_data["linear_acceleration_Z"] = normed_Z
    pd_data["linear_acceleration_Y"] = normed_Y
    
    # 下面对于相关性进行计算，删掉相关性小的字段
    pd_data.drop("linear_acceleration_X", axis=1, inplace=True)
    pd_data.drop("angular_velocity_X", axis=1, inplace=True)
    pd_data.drop("linear_acceleration_Z", axis=1, inplace=True)    
    print("删除相关性低的列之后... \n", pd_data.head(3))
    
    pd_data_sorted = pd_data.sort_values(by=["surface"])
    sample_train = pd_data_sorted.values[:, 2:]
    #print(sample_train[:3, :])
    totalCount = shape(sample_train)[0]
    print("样本总数为: %s, 0.6分割点: %s" % (totalCount, int(totalCount * 0.6)))
    
    return sample_train

getSampleData()[:2, :]

pd_data样本: 
           row_id  measurement_number  orientation_X  orientation_Y  \
series_id                                                            
0            0_0                   0       -0.75853       -0.63435   
0            0_1                   1       -0.75853       -0.63434   
0            0_2                   2       -0.75853       -0.63435   

           orientation_Z  orientation_W  angular_velocity_X  \
series_id                                                     
0               -0.10488       -0.10597            0.107650   
0               -0.10490       -0.10600            0.067851   
0               -0.10492       -0.10597            0.007275   

           angular_velocity_Y  angular_velocity_Z  linear_acceleration_X  \
series_id                                                                  
0                    0.017561            0.000767               -0.74857   
0                    0.029939            0.003385                0.33995   
0               

array([[-0.65046, 0.74478, 0.10758, -0.10315, 0.13747, -0.50052,
        0.6445053419572437, 7, 'carpet'],
       [-0.9636899999999999, 0.22115, 0.023597, -0.1478,
        0.20415999999999998, -0.44864, 0.5869294285802424, 7, 'carpet']],
      dtype=object)

In [67]:
# # 读入数据，加载到Pandas里面
# start_time = time.time()
# pd_x = pd.read_csv("x_train.csv")
# end_time = time.time()
# #print("pandas阅读csv文件耗时: ", end_time - start_time)
# pd_y = pd.read_csv("y_train.csv")
# pd_data = pd_x.set_index("series_id").join(pd_y.set_index("series_id"))
# print(pd_data.head(3))
# # 分析一下相关性，但是发现除了“measurement_number，linear_acceleration_X，angular_velocity_X，linear_acceleration_Z”
# # 无关之外其他字段还好
# corr_mastrix = pd_data.corr()
# corr_mastrix
# corr_mastrix["group_id"].sort_values(ascending=False)

In [68]:
# 统计各个分类的数量，用于后续划分测试集合和验证集合
def getClassCount(sample_train):
    classCount = {}

    for sample in sample_train:
        className = sample[-1]
        classCount[className] = classCount.get(className, 0) + 1

    return classCount

sample_train = getSampleData()
getClassCount(sample_train)

pd_data样本: 
           row_id  measurement_number  orientation_X  orientation_Y  \
series_id                                                            
0            0_0                   0       -0.75853       -0.63435   
0            0_1                   1       -0.75853       -0.63434   
0            0_2                   2       -0.75853       -0.63435   

           orientation_Z  orientation_W  angular_velocity_X  \
series_id                                                     
0               -0.10488       -0.10597            0.107650   
0               -0.10490       -0.10600            0.067851   
0               -0.10492       -0.10597            0.007275   

           angular_velocity_Y  angular_velocity_Z  linear_acceleration_X  \
series_id                                                                  
0                    0.017561            0.000767               -0.74857   
0                    0.029939            0.003385                0.33995   
0               

{'carpet': 24192,
 'concrete': 99712,
 'fine_concrete': 46464,
 'hard_tiles': 2688,
 'hard_tiles_large_space': 39424,
 'soft_pvc': 93696,
 'soft_tiles': 38016,
 'tiled': 65792,
 'wood': 77696}

In [69]:
from numpy import array

def splitTrainAndValidateData(sample_train, classCount):
    # 获取训练集合以及验证集合
    data_x = []
    data_y = []
    data_x_train = []
    data_y_train = []
    data_x_validate = []
    data_y_validate = []

    target = ""
    seria_count = 0
    validate_sample_count = 100

    for sample in sample_train:
        #print(sample)
        x = sample[:-2]
        data_x.append(x)
        y = sample[-1]
        data_y.append(y)

        if y != target:
            target = y
            seria_count = 0
            validate_sample_count = int(classCount[target]*0.6)
            #print("class: [%s] validate count: %d " % ( target, validate_sample_count))
        seria_count += 1
        if(seria_count <= validate_sample_count):
            #print("target is: %s, seria_count is: %s, branch: train" % (target, seria_count))
            data_x_train.append(x)
            data_y_train.append(y)
        else:
            #print("target is: %s, seria_count is: %s, branch: validate" % (target, seria_count))
            data_x_validate.append(x)
            data_y_validate.append(y)

    data_x_train_array = array(data_x_train)
    data_y_train_array = array(data_y_train)
    data_x_validate_array = array(data_x_validate)
    data_y_validate_array = array(data_y_validate)

    print("data_x_train_array shape: ", shape(data_x_train_array))
    print("data_y_train_array shape: ", shape(data_y_train_array))
    print("data_x_validate_array shape: ", shape(data_x_validate_array))
    print("data_y_validate_array shape: ", shape(data_y_validate_array))
    
    return data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array

sample_train = getSampleData()
print("sample_train: \n", sample_train[:2, :])
classCount = getClassCount(sample_train)
data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array = splitTrainAndValidateData(sample_train, classCount)
print(data_x_train_array[:2, :])

pd_data样本: 
           row_id  measurement_number  orientation_X  orientation_Y  \
series_id                                                            
0            0_0                   0       -0.75853       -0.63435   
0            0_1                   1       -0.75853       -0.63434   
0            0_2                   2       -0.75853       -0.63435   

           orientation_Z  orientation_W  angular_velocity_X  \
series_id                                                     
0               -0.10488       -0.10597            0.107650   
0               -0.10490       -0.10600            0.067851   
0               -0.10492       -0.10597            0.007275   

           angular_velocity_Y  angular_velocity_Z  linear_acceleration_X  \
series_id                                                                  
0                    0.017561            0.000767               -0.74857   
0                    0.029939            0.003385                0.33995   
0               

In [70]:
def fitAndValidate(ctl, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array):
    # 训练
    ctl.fit(data_x_train_array, data_y_train_array)
    # 预测
    y_predict = ctl.predict(data_x_validate_array)
    size = len(y_predict)
    # 统计错误率
    ok_count = 0
    error_count  = 0
    for index in range(size):
        if y_predict[index] == data_y_validate_array[index]:
            ok_count +=1
        else:
            error_count += 1
    # 输出结果
    print("数据总量%d，错误数：%d，错误率：%f" %(size, error_count, error_count/size))
    scores = cross_val_score(ctl, data_x_train_array, data_y_train_array, cv=5, scoring="accuracy")
    print("cross_val_score: ", scores)

In [71]:
# 获取训练数据集
def getData():
    sample_train = getSampleData()
    classCount = getClassCount(sample_train)
    return splitTrainAndValidateData(sample_train, classCount)

data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array = getData()

pd_data样本: 
           row_id  measurement_number  orientation_X  orientation_Y  \
series_id                                                            
0            0_0                   0       -0.75853       -0.63435   
0            0_1                   1       -0.75853       -0.63434   
0            0_2                   2       -0.75853       -0.63435   

           orientation_Z  orientation_W  angular_velocity_X  \
series_id                                                     
0               -0.10488       -0.10597            0.107650   
0               -0.10490       -0.10600            0.067851   
0               -0.10492       -0.10597            0.007275   

           angular_velocity_Y  angular_velocity_Z  linear_acceleration_X  \
series_id                                                                  
0                    0.017561            0.000767               -0.74857   
0                    0.029939            0.003385                0.33995   
0               

### knn

In [72]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(n_neighbors=1)
fitAndValidate(knn, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array)

# 未降维之前
# 数据总量195076，错误数：13042，错误率：0.066856

# 去掉了measurement_number列之后，
# 数据总量195076，错误数：1238，错误率：0.006346

# 对于“linear_acceleration_Y”和“linear_acceleration_Z”归一化之后
# 数据总量195076，错误数：80，错误率：0.000410

# 降维后（删掉了不相关的列之后）
# 数据总量195076，错误数：0，错误率：0.000000

数据总量195076，错误数：59778，错误率：0.306434
cross_val_score:  [0.67861048 0.7018659  0.73263841 0.68193236 0.66510587]


### Decision Tree

In [73]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=12, random_state=42)
fitAndValidate(tree, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array)

cross_val_score(tree, data_x_train_array, data_y_train_array, cv=5, scoring="accuracy")

# 未降维之前
# 数据总量195076，错误数：2527，错误率：0.012954

# 去掉了measurement_number列之后，
# 数据总量195076，错误数：1928，错误率：0.009883

# 对于“linear_acceleration_Y”和“linear_acceleration_Z”归一化之后
# 数据总量195076，错误数：1928，错误率：0.009883

# 降维后（删掉了不相关的列之后）
# 数据总量195076，错误数：2768，错误率：0.014189

数据总量195076，错误数：56265，错误率：0.288426
cross_val_score:  [0.67922562 0.72705215 0.73292891 0.69177532 0.68528804]


array([0.67922562, 0.72705215, 0.73292891, 0.69177532, 0.68528804])

### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=300,max_depth=15, max_leaf_nodes=15, n_jobs=-1)
fitAndValidate(forest, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array)
# 未降维前
# 数据总量195076，错误数：2620，错误率：0.013431

# 去掉了measurement_number列之后，
# 数据总量195076，错误数：1928，错误率：0.009883

# 对于“linear_acceleration_Y”和“linear_acceleration_Z”归一化之后
# 数据总量195076，错误数：1928，错误率：0.009883

# 降维后（删掉了不相关的列之后）
# 数据总量195076，错误数：2768，错误率：0.014189

数据总量195076，错误数：89952，错误率：0.461113
cross_val_score:  [0.56544324 0.56079557 0.58550923 0.56180044 0.53500692]


### 总结
### knn对于归一化还是比较敏感，但是下面的tree和randomForest则发现对于归一化没有反应；但是大部分的算法对于measurement_number这一列去掉之后的反应还是很明显的，这一列就是一个自增长，和分类毫无关联

In [ ]:
import time

start_time = time.time()
pd_x_test = pd.read_csv("X_test.csv")
# 对数据列进行归一化
max_min_scalar = lambda x: (x - np.min(x))/(np.max(x) - np.min(x))
normed_Z = pd_x_test[["linear_acceleration_Z"]].apply(max_min_scalar) # 注意这里一定是套了两层中括号
normed_Y = pd_x_test[["linear_acceleration_Y"]].apply(max_min_scalar)
pd_x_test["linear_acceleration_Z"] = normed_Z
pd_x_test["linear_acceleration_Y"] = normed_Y

# 下面对于相关性进行计算，删掉相关性小的字段
pd_x_test.drop("linear_acceleration_X", axis=1, inplace=True)
pd_x_test.drop("angular_velocity_X", axis=1, inplace=True)
pd_x_test.drop("linear_acceleration_Z", axis=1, inplace=True)

# 获取训练数据（跳过无关列）
data_x_test = pd_x_test.values[:, 3:]
#print(data_x_test[:3, :])
# 基于tree模型进行预测
predict_y = tree.predict(data_x_test)
# 遍历所有test样本来分析一下各个session的分类情况
sample_size = len(predict_y)
session = 0
sessionClassCount = {}
sessionClass = []
for i in range(sample_size):
    sample = pd_x_test.values[i, :]
    current_session = sample[1]
    class_label = predict_y[i]
    # 如果还是和上次session_id一样，则直接找到分类，添加次数即可
    if(current_session == session):
        sessionClassCount[class_label] = sessionClassCount.get(class_label, 0) + 1
    # 如果上次session_id不一样，则直接首先要统计一下上轮session_id的数据，选择出分类数量最多的分类作为整个session的分类
    # 记录参数进行初始化
    else:
        # 处理上轮session数据统计
        max_class = sorted(sessionClassCount, key=lambda p:p[1], reverse=True)[0]
        sessionClass.append(max_class)
        print("session %d: %s" %(session, max_class))
        # 对记录参数进行初始化
        sessionClassCount = {}
        session = current_session
        # 记录新一轮第一个分类数据
        sessionClassCount[class_label] = 1
        
end_time = time.time()
diff = end_time - start_time
print("总共花费时间: ", diff)
# ss = pd.read_csv("sample_submission.csv")
# ss.head(3)
# pd_x_test.head(3)
# ss["surface"] = predict_y
# ss.to_csv("sample_submission.csv")

session 0: fine_concrete
session 1: concrete
session 2: soft_pvc
session 3: wood
session 4: soft_pvc
session 5: concrete
session 6: concrete
session 7: wood
session 8: concrete
session 9: wood
session 10: concrete
session 11: soft_pvc
session 12: hard_tiles_large_space
session 13: concrete
session 14: soft_pvc
session 15: concrete
session 16: concrete
session 17: fine_concrete
session 18: wood
session 19: soft_pvc
session 20: concrete
session 21: concrete
session 22: concrete
session 23: hard_tiles_large_space
session 24: concrete
session 25: wood
session 26: concrete
session 27: soft_tiles
session 28: concrete
session 29: concrete
session 30: wood
session 31: soft_tiles
session 32: concrete
session 33: soft_pvc
session 34: concrete
session 35: concrete
session 36: fine_concrete
session 37: concrete
session 38: soft_pvc
session 39: soft_tiles
session 40: wood
session 41: soft_pvc
session 42: soft_pvc
session 43: wood
session 44: soft_pvc
session 45: carpet
session 46: concrete
session 

session 374: tiled
session 375: wood
session 376: soft_tiles
session 377: tiled
session 378: concrete
session 379: soft_pvc
session 380: soft_tiles
session 381: soft_pvc
session 382: wood
session 383: fine_concrete
session 384: tiled
session 385: hard_tiles_large_space
session 386: soft_pvc
session 387: concrete
session 388: soft_pvc
session 389: carpet
session 390: soft_tiles
session 391: hard_tiles_large_space
session 392: concrete
session 393: soft_tiles
session 394: concrete
session 395: concrete
session 396: concrete
session 397: soft_pvc
session 398: concrete
session 399: hard_tiles_large_space
session 400: wood
session 401: wood
session 402: soft_pvc
session 403: concrete
session 404: concrete
session 405: concrete
session 406: concrete
session 407: soft_pvc
session 408: wood
session 409: soft_pvc
session 410: wood
session 411: wood
session 412: concrete
session 413: soft_pvc
session 414: wood
session 415: wood
session 416: soft_pvc
session 417: soft_pvc
session 418: concrete
se

session 747: concrete
session 748: concrete
session 749: fine_concrete
session 750: concrete
session 751: concrete
session 752: wood
session 753: soft_pvc
session 754: soft_pvc
session 755: concrete
session 756: concrete
session 757: soft_tiles
session 758: fine_concrete
session 759: concrete
session 760: wood
session 761: soft_tiles
session 762: concrete
session 763: wood
session 764: concrete
session 765: hard_tiles_large_space
session 766: wood
session 767: concrete
session 768: concrete
session 769: fine_concrete
session 770: soft_pvc
session 771: concrete
session 772: wood
session 773: soft_pvc
session 774: tiled
session 775: soft_pvc
session 776: concrete
session 777: concrete
session 778: soft_pvc
session 779: concrete
session 780: soft_tiles
session 781: concrete
session 782: concrete
session 783: hard_tiles_large_space
session 784: soft_tiles
session 785: soft_pvc
session 786: soft_pvc
session 787: wood
session 788: soft_tiles
session 789: concrete
session 790: wood
session 79

session 1108: wood
session 1109: soft_pvc
session 1110: wood
session 1111: soft_tiles
session 1112: soft_pvc
session 1113: soft_tiles
session 1114: fine_concrete
session 1115: soft_tiles
session 1116: concrete
session 1117: concrete
session 1118: concrete
session 1119: concrete
session 1120: carpet
session 1121: fine_concrete
session 1122: fine_concrete
session 1123: soft_pvc
session 1124: tiled
session 1125: concrete
session 1126: concrete
session 1127: hard_tiles_large_space
session 1128: soft_tiles
session 1129: concrete
session 1130: soft_tiles
session 1131: concrete
session 1132: concrete
session 1133: carpet
session 1134: concrete
session 1135: soft_pvc
session 1136: tiled
session 1137: wood
session 1138: soft_pvc
session 1139: concrete
session 1140: hard_tiles_large_space
session 1141: concrete
session 1142: soft_pvc
session 1143: wood
session 1144: soft_pvc
session 1145: soft_pvc
session 1146: concrete
session 1147: concrete
session 1148: soft_pvc
session 1149: soft_pvc
session

session 1459: soft_pvc
session 1460: fine_concrete
session 1461: soft_pvc
session 1462: soft_pvc
session 1463: concrete
session 1464: wood
session 1465: tiled
session 1466: wood
session 1467: concrete
session 1468: soft_pvc
session 1469: fine_concrete
session 1470: soft_pvc
session 1471: concrete
session 1472: hard_tiles_large_space
session 1473: concrete
session 1474: hard_tiles_large_space
session 1475: wood
session 1476: wood
session 1477: fine_concrete
session 1478: soft_pvc
session 1479: concrete
session 1480: wood
session 1481: wood
session 1482: concrete
session 1483: soft_pvc
session 1484: hard_tiles_large_space
session 1485: concrete
session 1486: concrete
session 1487: wood
session 1488: tiled
session 1489: wood
session 1490: wood
session 1491: concrete
session 1492: soft_pvc
session 1493: concrete
session 1494: soft_pvc
session 1495: soft_pvc
session 1496: wood
session 1497: soft_pvc
session 1498: soft_pvc
session 1499: hard_tiles_large_space
session 1500: hard_tiles_large_s

session 1808: concrete
session 1809: concrete
session 1810: concrete
session 1811: concrete
session 1812: concrete
session 1813: soft_pvc
session 1814: concrete
session 1815: soft_pvc
session 1816: hard_tiles_large_space
session 1817: wood
session 1818: soft_tiles
session 1819: wood
session 1820: wood
session 1821: fine_concrete
session 1822: concrete
session 1823: fine_concrete
session 1824: soft_pvc
session 1825: wood
session 1826: concrete
session 1827: tiled
session 1828: concrete
session 1829: soft_pvc
session 1830: soft_tiles
session 1831: concrete
session 1832: wood
session 1833: wood
session 1834: soft_pvc
session 1835: tiled
session 1836: concrete
session 1837: soft_tiles
session 1838: concrete
session 1839: soft_pvc
session 1840: concrete
session 1841: soft_pvc
session 1842: soft_pvc
session 1843: wood
session 1844: wood
session 1845: concrete
session 1846: tiled
session 1847: wood
session 1848: soft_pvc
session 1849: hard_tiles_large_space
session 1850: tiled
session 1851: c

session 2162: concrete
session 2163: hard_tiles_large_space
session 2164: concrete
session 2165: soft_pvc
session 2166: soft_pvc
session 2167: concrete
session 2168: soft_pvc
session 2169: tiled
session 2170: soft_tiles
session 2171: concrete
session 2172: soft_pvc
session 2173: soft_tiles
session 2174: wood
session 2175: soft_pvc
session 2176: concrete
session 2177: wood
session 2178: fine_concrete
session 2179: soft_pvc
session 2180: concrete
session 2181: tiled
session 2182: hard_tiles_large_space
session 2183: wood
session 2184: wood
session 2185: tiled
session 2186: hard_tiles_large_space
session 2187: wood
session 2188: hard_tiles_large_space
session 2189: fine_concrete
session 2190: tiled
session 2191: concrete
session 2192: soft_pvc
session 2193: wood
session 2194: soft_tiles
session 2195: soft_tiles
session 2196: concrete
session 2197: soft_pvc
session 2198: soft_tiles
session 2199: hard_tiles_large_space
session 2200: soft_tiles
session 2201: wood
session 2202: concrete
sessi

session 2509: wood
session 2510: concrete
session 2511: concrete
session 2512: wood
session 2513: soft_tiles
session 2514: concrete
session 2515: wood
session 2516: concrete
session 2517: wood
session 2518: fine_concrete
session 2519: tiled
session 2520: concrete
session 2521: concrete
session 2522: soft_pvc
session 2523: concrete
session 2524: wood
session 2525: concrete
session 2526: concrete
session 2527: fine_concrete
session 2528: concrete
session 2529: tiled
session 2530: concrete
session 2531: soft_tiles
session 2532: fine_concrete
session 2533: hard_tiles_large_space
session 2534: concrete
session 2535: soft_pvc
session 2536: wood
session 2537: soft_tiles
session 2538: concrete
session 2539: concrete
session 2540: fine_concrete
session 2541: concrete
session 2542: wood
session 2543: hard_tiles_large_space
session 2544: tiled
session 2545: wood
session 2546: concrete
session 2547: soft_pvc
session 2548: concrete
session 2549: soft_pvc
session 2550: soft_tiles
session 2551: soft_